In [ ]:
#LAUNCH JUPYER WITH COMMAND BELOW OR IT MAY HAVE DIFFICULTY TO EXECUTE THE CODE

jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000

In [ ]:
#LIBRARIES:
#networkx : is a library used to create directional verteces that will be used in the Genetic Algorithm for optimization later
#numpy : for array management
#pandas : to load *.csv and convert data
#time : to calculate time between iteration and global computation time
#matplotlib.pyplot : for graphs
#copy : for true copy of variables and arrays
#heapq : used in main Loop in order to select best Chromosome from Genetic Algorithm iterations
#math : for calculation

In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import random
import copy
import heapq
import math

In [ ]:
#ORIGINAL FILE WAS CONVERTED INTO DIFFERENT *.csv files
#"project.csv" this is the table with all the connections between stations and the trip duration between
#"nameTable" : table with the name of the stations and the index code seen in project.csv
#"outsideParis" : list of the stations beyond the border of paris ("Mairie d'Ivry" for example)
#insideParis" : list of the stations inside paris
#There are 376 rows in project corresponding to connections between 2 stations
#There are 296 single name stations
#There are 54 stations outside Paris
#Finally, Inside Paris there 242 different stations with all a different name

In [ ]:
df = pd.read_csv('C:/Projects IT/Python/Projet Metaheuristic/project_correction.csv', sep =';')
nameTable = pd.read_csv('C:/Projects IT/Python/Projet Metaheuristic/indexStation.csv', sep =';')
outsideParis = pd.read_csv('C:/Projects IT/Python/Projet Metaheuristic/indexStation_outsideParis.csv', sep =';')
insideParis = pd.read_csv('C:/Projects IT/Python/Projet Metaheuristic/indexStationParis_corrected.csv', sep =';')

df.tail(3)

In [ ]:
df.dtypes

In [ ]:
len(np.unique(insideParis.nameStation))

In [ ]:
nameTable.head()

In [ ]:
outsideParis.head(3)

In [ ]:
#CREATION OF THE GRAPH with networkx, I use a Multiple directed graph, 
# it means that a node can have multiple connections with different nodes and the direction matters
#for example 34 connect with 248 but 248 doesnt connect with 34

In [ ]:
MG = nx.MultiDiGraph()

In [ ]:
#CREATION OF THE EDGES
#using the df table I create the edges inside the Graph named MG (see above), 
#the weight will be the Time spent between two stations

In [ ]:
for i in range(0, len(df.Station)):
    MG.add_edge(df.Station[i], df.Connection[i], weight=df.Time[i])

In [ ]:
MG

In [ ]:
#THE EDGES ARE REPRESENTED BY A DICTIONNARY

In [ ]:
print(MG.adj[0])

In [ ]:
#networkx can plot a graph with connections and directions between different nodes

In [ ]:
plt.figure(figsize=(12,12))
nx.draw(MG, label=False)

plt.show()

In [ ]:
#The Johnson's algorithm will give shortest paths between stations for every verteces
#I used it because it's fast, and usable with weight and directions

https://en.wikipedia.org/wiki/Johnson%27s_algorithm

In [ ]:
shortestMulti = nx.johnson(MG)

In [ ]:
#EXAMPLE OF A SHORTEST PATH BETWEEN STATION 0 AND 375 thanks to the Johnson's Algorithm
shortestMulti[0][375]

In [ ]:
#MAIN CLASS DEFINITION: Chromosome, it represents a set of nodes(genes).
#******-------------------------------------------------------------------******
#the class function __ini__ is the constructor of the class, it demands a set of stations(index)
#----------------------------------------------------------------------------
#the shuffleGenes is a function in order to shuffle the nodes/genes in the chromosome
#-------------------------------------------------------------------------------
#mutationGene : is a fuction that permits to mutate some gene in the sequence of the chromosome, thanks to a probability  Pm.
#the function "reads" the chromosome from the beginning, nade after node, then a probability is done (randomly), if it is below
#a Threshold Pm that gene will be mutated
#Mutation adds some randomness to a chromosome and brings some variety into the population
#---------------------------------------------------------------------------------
#pathWay : This function creates the optimal pathway between 2 nodes using the dictionnary created with networkx (see above)
#For example between 2 nodes (3,47) the optimal pathway calculated by Johnson's algorithm 
#will be [3, 210, 95, 292, 361, 208, 333, 334, 323, 222, 221, 74, 195, 47]
#THE CHROMOSOME IS ENCODED LIKE THAT
#Chromosome  = |23|45|156|176 etc..... and pathway give the optimal path from 23 to 45, 45 to 156, 156 to 176...... 
#as a numpy.array
#This way, the Genetic Algorithm can optimize the "genes" and there will always be a link between stations
#-----------------------------------------
#__repr__ : it's a function to overload the print function for an object of Chromosome's class
#example
#--------------------
#test = Chromosome([1,2,3,4,5])
#print(test)
#output >> Chromosome :|1|2|3|4|5|

In [ ]:
class Chromosome:
    def __init__(self, nodes):
        self.nodes = nodes       
    
    def shuffleGenes(self):
        random.shuffle(self.nodes)   
    
    #Order Changing Mutation, 1 gene is selected thanks to Pm probability
    def mutationGene(self, Pm, inParis):
        self.inParis = inParis
        self.Pm = Pm
        for i in range(0, len(self.nodes)):
            self.randMutate = random.random()
            if self.randMutate < self.Pm:
                self.indexMutation = random.randint(0,len(self.inParis)-1)
                self.mutateTo = self.inParis[self.indexMutation]
                self.nodes[i] = self.mutateTo


    #each gene represents a shortest path between gene i and gene i+1
    def pathWay(self, Multigraph): #Multigraph comes from the Johnson Networkx calculation for shortest path between 2 nodes
        self.Multigraph = Multigraph
        self.totalPath = []
        for i in range(0, len(self.nodes)-1):        
            if i+1 is not None:
                self.totalPath = merge(self.totalPath, self.Multigraph[self.nodes[i]][self.nodes[i+1]])   
        return self.totalPath       
        
    def __repr__(self):
        self.strNodes = "Chromosome :"
        for i in self.nodes:
            self.node = str(i)
            self.strNodes = self.strNodes + "|" + self.node
        self.strNodes = self.strNodes + "|"
        return self.strNodes

In [ ]:
#FUNCTION PathwayDuration : calculate the path time for a chromosome
#-----------------------------------------------------------------
#PARAMETERS :
#chromosome : the object of class Chromosome to use
#multipath : the Dictionary of shortest path between nodes
#mainDataframe : dataframe with connection between stations and traveling time (here it is called "df")
#Returns the total time in seconds shouldnt be > 72000 (20 hours from 5AM to 1AM)

In [ ]:
def PathwayDuration(chromosome, multipath, mainDataframe):
    b = 0
    for i in range(1, len(chromosome.pathWay(multipath))):
        a = np.array(df['Time'].loc[(mainDataframe.Station == chromosome.pathWay(multipath)[i-1]) & 
                               (mainDataframe.Connection == chromosome.pathWay(multipath)[i])].values)
        b = b + a 
    return int(b)

In [ ]:
#FUNCTION crossoverSinglePoint:
#A function that returns 1 child chromosome from 2 parents chromosomes
#2 chromosomes are splitted halfway then for example
#Chrom 1 = A|B|C|D and chrom2 = E|F|G|H then we obtain : A|B + G|H
#Child1 = A|B|G|H and if you permute chromosomes in a second call you obtain child2 = E|F|C|D

In [ ]:
def crossoverSinglePoint(chromo1, chromo2):
    lenChromo = int(len(chromo1.nodes) / 2)
    childrenChr = np.append(chromo1.nodes[0:lenChromo], chromo2.nodes[lenChromo:len(chromo1.nodes)])
    return childrenChr    

In [ ]:
#FUNCTION merge:
#works with np.array
#function can merge 2 list where last element of one(a) is the first element of the second(b)
#for example [1,2,3,4] + [4,5,6] = [1,2,3,4,5,6]
#

In [ ]:
def merge(a, b): #a,b are numpy.array
    c = a[:-1] + b[:]
    return c    

In [ ]:
#FUNCTION NewNameFit2
#return the score to maximize, 1 = best answer, it means that all stations with a different name, have been visited
#There are 242 stations inside paris with a different name, I divide unique name from the list 
#of the nodes by 242,in order to obtain the score.
#the len() is here to prevent the pathway to go beyond 1200 stations visited 
#that could take more than 20 hours to visit all
#Remark : fitScore = 0 if there is a station visited if the length of the chromosome goes beyond 1200(too long)

In [ ]:
def NewNameFit2(uniqueNodeName, chromosome, MultiG, uniqueNodeIndex):
    if len(chromosome.pathWay(MultiG)) < 1200:
        NameList = np.array([])
        for i in chromosome.pathWay(MultiG):
            a, = np.where(uniqueNodeIndex == i)
            NameList = np.append(NameList, uniqueNodeName[a])
            fitScore = len(np.unique(NameList)) / 242 #number of unique station / total unique station(242)
    else:
        fitScore = 0 #score equals 0 if the pathway is too long
    return fitScore

In [ ]:
#***********________________________________________******************
#***********___MAIN LOOP OF THE GENETIC ALGORITHM___******************
#***********________________________________________******************
#First a population is generated, a population is a set of chromosome with a sequence of different nodes/genes for each
#population is an array of objects of class Chromosome()
#population will evolve at each iteration of the genetic algorithm
#*****-----------------------------------------------------------------
#Then, from the initial population 2(could be changed) best scores are kept, 
#and From the 10(can be changed) best candidates 18 children are created with the crossover Function according to the function :
# ABCD and EFGH gives ABGH, EFCD,  
#each gene can be mutated with Pm probability, 
#for example if the chromosome has a length of 100 and the probability for a gene to mutate is
#Pm = 0.02 there will an average of 2 genes mutated, can be lower or higher depends of the value of Pm
#***********------------------------------------------------------------
#FINALLY, we obtain a new population composed of Chrildren Chromosome and 
#the 2 best Chromosomes from the former population and each
#chromosome could be mutated (depending of Pm)
#Then the new population will have a new iteration etc.....

In [ ]:
#NEW ALGO split avec duplicate CROSSOVER SINGLE POINT
#MAIN LOOP
population = np.array([]) #initialization of the population array
X_opti = np.array([]) #initialization of X axis for the graph
Y_opti = np.array([]) #initialization of Y axis for the graph
fitMax = 0
#CREATION OF THE INITIAL POPULATION
for i in range(0, 100):
    tmpNode = np.random.choice(insideParis.indexStation, 80, replace=False)
    tmpC = Chromosome(tmpNode)
    population = np.append(population, tmpC)
#MAIN LOOP here population evolved continuously with crossovers and mutations
#NUMBER OF ITERATION z
for z in range(0,200):
    fitList = np.array([])
    
    #fitList = array of the score of the population
    for i in range(0,len(population)):
        fitList = np.append(fitList, NewNameFit2(insideParis.nameStation, population[i], shortestMulti, insideParis.indexStation))
        if fitList[i] > fitMax:
            fitMax = fitList[i]
            bestSolution = population[i] #selection of the best candidate
    if fitMax > 0.999:
            break
    #select the 10 best chromosomes according to their scores, 
    #the array is ordered from higher to lower score, give 18 children
    IndexfitBest = heapq.nlargest(10, range(len(fitList)), fitList.take)
    population_best = np.take(population, IndexfitBest)
    
    #initialization of the new population with 2 best scored chromosomes
    child = np.array([population_best[0], population_best[1]]) #2 best chromosomes kept
    
    #Creation of the children :
    for u in range(1, len(population_best)):
        crossChr1 = Chromosome(crossoverSinglePoint(population_best[u-1], population_best[u]))
        crossChr2 = Chromosome(crossoverSinglePoint(population_best[u], population_best[u-1]))
        child = np.append(child, crossChr1)
        child = np.append(child, crossChr2)
    population = child
    
    #Mutation
    for mutIter in range(0, len(population)-1):
        rand = random.random()
        if rand < 1:
            population[mutIter].mutationGene(0.02, insideParis.indexStation) # a mutation has 10% chance to occur
    print(np.max(fitList)) #Print the best score of the best candidate
    X_opti = np.append(X_opti, z) #keep track of iterations
    Y_opti = np.append(Y_opti, np.max(fitList)) #keep track of the best score obtained
print(bestSolution)
print(population)


In [ ]:
#Thanks to X_opti and Y_opti we can draw the graph
#With the number of iterations on the X axis and the number of different stations visited on the Y axis
plt.plot(X_opti, Y_opti * 242) #Y axis = maximum name visited
plt.show()

In [ ]:
fitList

In [ ]:
print(PathwayDuration(population[1],shortestMulti, df )/3600)

In [ ]:

population[0].pathWay(shortestMulti)


In [ ]:
NewNameFit2(insideParis.nameStation,bestSolution, shortestMulti, insideParis.indexStation)